# Image Classification using convolutional layers with Keras
In this notebook, a Convolutional Neural Network will be build from scratch and trained on the MNIST dataset. The MNIST dataset is a dataset containing hand-written images of digits. The end goal is to label the images with their corresponding digit. Take a look at the Feedforward Neural Network Image Classification of the MNIST dataset for the exploring of the MNIST dataset and the explanation about the data preparation.    

In [1]:
# Imports
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

## Dataset preparation
First, the MNIST data is loaded from the Keras dataset library. It directly loads in a training and test dataset. X denotes the images and y denotes the labels. We can also have a look at the size of these datasets. 

In [2]:
# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Print results
print('The train dataset contains ' + str(x_train.shape[0]) + ' samples.')
print('The test dataset contains ' + str(x_test.shape[0]) + ' samples.')

The train dataset contains 60000 samples.
The test dataset contains 10000 samples.


Next, the images are converted to doubles and normalized. Initially, the images have pixel values ranging from 0 to 255 (8 bits). But for the network to train faster, it is always beneficial to normalize the input values between 0 and 1.

In [3]:
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# Model / data parameters
num_classes = 10

The labels of the dataset are numbers from 0 to 9. It is common practice in neural network training to convert these numbers to binary vectors  with a 1 at the index of the correct number and zeros everywhere else. 

In [4]:
# The labels of the dataset are the correct numbers for each image
print("The label of the first image is: " + str(y_train[0]))

# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# The labels are converted to binary vectors
print("The new label of the first image is: " + str(y_train[0]))

The label of the first image is: 5
The label of the first image is: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


## Creating the neural network model
As a neural network, we use a sequence of 2 convolutional layers followed by a pooling layer. The first convolutional layer contains 32 filters with size 3x3 and has a ReLu function as activation. The first convolutional layer contains 64 filters with size 3x3 and also has a ReLu function as activation. These two convolutional layers together are the feature extraction part of the neural network. A dropout layer is included to prevent overfitting. After these layers there is a fully connected (dense) layer that represents the classification part of the neural network. This final layer outputs 10 neurons that each represent a class instance, being a number from 0 to 9. The activation is a SoftMax layer that outputs a probability for each of the classes.  

In [7]:
# Defines the neural network model
model = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

# Summarizes the model and its parameters
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

## Model training
Before we can train the model, we need to define what loss, optimizer, and accuracy metric we want to use. As we are dealing with a multi-class classification problem, we use the categorical cross entropy loss. As optimizer we use the popular Adam algorithm. And we will define accuracy as the metric of goodness for our network. We also define that we will validate the network after every epoch on a dataset that has 10% of the samples in the training dataset that was defined earlier. 

In [8]:
# Training parameters
epochs = 15
batch_size = 128

# Define the loss function, optimizer, and accuracy metric
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 35s 79ms/step - loss: 0.3643 - accuracy: 0.8919 - val_loss: 0.0871 - val_accuracy: 0.9753
Epoch 2/15
422/422 [==============================] - 33s 79ms/step - loss: 0.1120 - accuracy: 0.9659 - val_loss: 0.0584 - val_accuracy: 0.9852
Epoch 3/15
422/422 [==============================] - 33s 79ms/step - loss: 0.0853 - accuracy: 0.9742 - val_loss: 0.0529 - val_accuracy: 0.9853
Epoch 4/15
422/422 [==============================] - 34s 82ms/step - loss: 0.0716 - accuracy: 0.9783 - val_loss: 0.0436 - val_accuracy: 0.9882
Epoch 5/15
422/422 [==============================] - 35s 83ms/step - loss: 0.0633 - accuracy: 0.9808 - val_loss: 0.0382 - val_accuracy: 0.9892
Epoch 6/15
422/422 [==============================] - 34s 81ms/step - loss: 0.0556 - accuracy: 0.9826 - val_loss: 0.0334 - val_accuracy: 0.9912
Epoch 7/15
422/422 [==============================] - 30s 72ms/step - loss: 0.0507 - accuracy: 0.9839 - val_loss: 0.0345 - val_accuracy:

## Model validation
When the model is trained, we can evalueate its performance on the test set. By calling the evaluate-function of the model it uses the earlier defined goodness metric for the evaluation. 

In [9]:
# Evaluate model
score = model.evaluate(x_test, y_test, verbose=0)

# Print results
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.023343771696090698
Test accuracy: 0.9911999702453613
